In [6]:
from make_dataset_helper import prepare_mnist
import tensorflow as tf
import numpy as np
import logging
import matplotlib.pyplot as plt
% matplotlib inline


def create_placeholder(X, y):
    X = tf.placeholder(name="X", dtype=tf.float32, shape=(None, X.shape[1]))
    y = tf.placeholder(name="y", dtype=tf.float32, shape=(None, y.shape[1]))
    return X, y


def initialize_parameter(n_dims):
    parameters = {}
    for i in range(1, len(n_dims)):
        parameters["W" + str(i)] = tf.get_variable(name="W"+str(i), shape=(n_dims[i], n_dims[i-1]), initializer=tf.contrib.layers.xavier_initializer())
        parameters["b" + str(i)] = tf.get_variable(name="b" + str(i), shape=(n_dims[i], 1), initializer=tf.zeros_initializer())
    return parameters


def forward_function(parameters, X, n_dims):
    outputs = {}
    outputs["A0"] = X
    for i in range(1, len(n_dims)):
        Z = tf.add(tf.matmul(parameters["W" + str(i)], outputs["A" + str(i - 1)]), parameters["b" + str(i)])
        A = tf.nn.relu(Z)
        outputs["Z" + str(i)] = Z
        if i ==len(n_dims) - 1:
            continue
        outputs["A" + str(i)] = A
    return Z
        

def compute_cost(ZL, y):
    logits = tf.transpose(ZL)
    labels = tf.transpose(y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return cost
        

def main(_logger, X_train, X_test, y_train, y_test,  n_labels, n_dims, learning_rate=0.01, iteration_num=100, cost_print=True):
    _logger.info("initialize the computation graph")
    tf.reset_default_graph()   
        
    _logger.info("create X, y placeholder")
    X, y = create_placeholder(X_train, y_train)
    
    _logger.info("intialize_parameter")
    parameters = initialize_parameter(n_dims)
    
    _logger.info("forward propagation")
    ZL = forward_function(parameters, X, n_dims)
    
    _logger.info("compute cost function")
    cost = compute_cost(ZL, y)
    
    _logger.info("optimize with optimizer")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # initialize all valuables
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        _logger.info("Begin optimizing cost")
        for epoch in range(1, iteration_num+1):
            cost_val, _ = sess.run([cost, optimizer],feed_dict={X: X_train, y: y_train})
            if cost_print==True and epoch % 10 == 0:
                print("epoch" + str(epoch), cost_val)
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        _logger.info("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        train_accuracy = accuracy.eval({X: X_train, y: y_train})
        test_accuracy = accuracy.eval({X: X_test, y: y_test})
        
        _logger.info("Train Accuracy: {}".format(train_accuracy))
        _logger.info("Test Accuracy: {}".format(test_accuracy))
        


if __name__ == '__main__':
    _logger = logging.getLogger(__name__)
    logging.basicConfig(
    level=logging.INFO, format="%(asctime)s\t%(levelname)s\t%(name)s\t%(message)s"
    )
    X_train, X_test, y_train, y_test, n_labels = prepare_mnist()
    n_dims = [X_train.shape[0], 30, n_labels]
    main(_logger, X_train, X_test, y_train, y_test,  n_labels, n_dims, learning_rate=0.01, iteration_num=100)
    
    

2018-10-01 10:55:45,382	INFO	__main__	initialize the computation graph
2018-10-01 10:55:45,384	INFO	__main__	create X, y placeholder
2018-10-01 10:55:45,389	INFO	__main__	intialize_parameter
2018-10-01 10:55:45,450	INFO	__main__	forward propagation
2018-10-01 10:55:45,459	INFO	__main__	compute cost function
2018-10-01 10:55:45,536	INFO	__main__	optimize with optimizer
2018-10-01 10:55:45,780	INFO	__main__	Begin optimizing cost


epoch10 2.5693407
epoch20 2.0060406
epoch30 1.9772935
epoch40 1.9243577
epoch50 1.8615102
epoch60 1.8052714
epoch70 1.7704896
epoch80 1.7391356
epoch90 1.7081258
epoch100 1.6812232


NameError: name 'costs' is not defined

In [ ]:
from make_dataset_helper import prepare_mnist, prepare_iris, prepare_digits
import tensorflow as tf
import numpy as np
import logging
import matplotlib.pyplot as plt
% matplotlib inline


def create_placeholder(X, y):
    X = tf.placeholder(name="X", dtype=tf.float32, shape=(X.shape[0], None))
    y = tf.placeholder(name="y", dtype=tf.float32, shape=(y.shape[0], None))
    return X, y


def initialize_parameter(n_dims):
    parameters = {}
    for i in range(1, len(n_dims)):
        parameters["W" + str(i)] = tf.get_variable(name="W"+str(i), shape=(n_dims[i], n_dims[i-1]), initializer=tf.contrib.layers.xavier_initializer())
        parameters["b" + str(i)] = tf.get_variable(name="b" + str(i), shape=(n_dims[i], 1), initializer=tf.zeros_initializer())
    return parameters


def forward_function(_logger, parameters, X, n_dims, activation):
    outputs = {}
    outputs["A0"] = X
    _logger.info("Activation function is {}".format(activation))
    for i in range(1, len(n_dims)):
        Z = tf.add(tf.matmul(parameters["W" + str(i)], outputs["A" + str(i - 1)]), parameters["b" + str(i)])
        
        if activation == "relu":
            A = tf.nn.relu(Z)
            
        elif activation == "sigmoid":
            A = tf.nn.sigmoid(Z)
            
        elif activation == "tanh":
            A = tf.nn.tanh(Z)
            
        outputs["Z" + str(i)] = Z
        if i ==len(n_dims) - 1:
            continue
        outputs["A" + str(i)] = A
    return Z
        

def compute_cost(ZL, y):
    logits = tf.transpose(ZL)
    labels = tf.transpose(y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return cost
        

def main(_logger, X_train, X_test, y_train, y_test,  n_labels, n_dims, learning_rate=0.01, iteration_num=100, cost_print=True, activation="relu"):
    _logger.info("initialize the computation graph")
    tf.reset_default_graph()   
        
    _logger.info("create X, y placeholder")
    X, y = create_placeholder(X_train, y_train)
    
    _logger.info("intialize_parameter")
    parameters = initialize_parameter(n_dims)
    
    _logger.info("forward propagation")
    ZL = forward_function(_logger, parameters, X, n_dims, activation)
    
    _logger.info("compute cost function")
    cost = compute_cost(ZL, y)
    
    _logger.info("optimize with optimizer")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # initialize all valuables
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        _logger.info("Begin optimizing cost")
        costs = []
        
        for epoch in range(1, iteration_num+1):
            cost_val, _ = sess.run([cost, optimizer],feed_dict={X: X_train, y: y_train})
            costs.append(cost_val)
            if cost_print==True and epoch % 10 == 0:
                print("epoch" + str(epoch), cost_val)
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        _logger.info("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(ZL), tf.argmax(y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        train_accuracy = accuracy.eval({X: X_train, y: y_train})
        test_accuracy = accuracy.eval({X: X_test, y: y_test})
        
        _logger.info("Train Accuracy: {}".format(train_accuracy))
        _logger.info("Test Accuracy: {}".format(test_accuracy))
        


if __name__ == '__main__':
    _logger = logging.getLogger(__name__)
    logging.basicConfig(
    level=logging.INFO, format="%(asctime)s\t%(levelname)s\t%(name)s\t%(message)s"
    )
    X_train, X_test, y_train, y_test, n_labels = prepare_mnist()
    n_dims = [X_train.shape[0], 30, n_labels]
    main(_logger, X_train, X_test, y_train, y_test,  n_labels, n_dims, learning_rate=0.01, iteration_num=100)

2018-10-01 11:12:04,931	INFO	__main__	initialize the computation graph
2018-10-01 11:12:04,932	INFO	__main__	create X, y placeholder
2018-10-01 11:12:04,936	INFO	__main__	intialize_parameter
2018-10-01 11:12:05,006	INFO	__main__	forward propagation
2018-10-01 11:12:05,009	INFO	__main__	Activation function is relu
2018-10-01 11:12:05,021	INFO	__main__	compute cost function
2018-10-01 11:12:05,147	INFO	__main__	optimize with optimizer
2018-10-01 11:12:05,505	INFO	__main__	Begin optimizing cost


epoch10 2.3682578
epoch20 2.0907528
epoch30 2.027593
epoch40 1.9643722
epoch50 1.9159491
epoch60 1.8703178
